In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
text = open('khayyam.txt', 'rb').read().decode(encoding='utf-8')

In [3]:
text[:10]

'|برخیز بتا'

In [4]:
vocabolaries = sorted(set(text))

In [5]:
vocabolaries

['\n',
 ' ',
 '!',
 '|',
 '؟',
 'ء',
 'آ',
 'ؤ',
 'ئ',
 'ا',
 'ب',
 'ة',
 'ت',
 'ث',
 'ج',
 'ح',
 'خ',
 'د',
 'ذ',
 'ر',
 'ز',
 'س',
 'ش',
 'ص',
 'ض',
 'ط',
 'ظ',
 'ع',
 'غ',
 'ف',
 'ق',
 'ل',
 'م',
 'ن',
 'ه',
 'و',
 'ً',
 'ٌ',
 'َ',
 'ّ',
 'ٔ',
 'پ',
 'چ',
 'ژ',
 'ک',
 'گ',
 'ی']

In [6]:
len(vocabolaries)

47

In [7]:
char2index = {u:i for i, u in enumerate(vocabolaries)}
index2char = np.array(vocabolaries)

In [8]:
char2index

{'\n': 0,
 ' ': 1,
 '!': 2,
 '|': 3,
 '؟': 4,
 'ء': 5,
 'آ': 6,
 'ؤ': 7,
 'ئ': 8,
 'ا': 9,
 'ب': 10,
 'ة': 11,
 'ت': 12,
 'ث': 13,
 'ج': 14,
 'ح': 15,
 'خ': 16,
 'د': 17,
 'ذ': 18,
 'ر': 19,
 'ز': 20,
 'س': 21,
 'ش': 22,
 'ص': 23,
 'ض': 24,
 'ط': 25,
 'ظ': 26,
 'ع': 27,
 'غ': 28,
 'ف': 29,
 'ق': 30,
 'ل': 31,
 'م': 32,
 'ن': 33,
 'ه': 34,
 'و': 35,
 'ً': 36,
 'ٌ': 37,
 'َ': 38,
 'ّ': 39,
 'ٔ': 40,
 'پ': 41,
 'چ': 42,
 'ژ': 43,
 'ک': 44,
 'گ': 45,
 'ی': 46}

In [9]:
index2char[1]

' '

In [10]:
text_as_integer = np.array([char2index[c] for c in text])

In [11]:
text_as_integer

array([ 3, 10, 19, ..., 31,  9, 32])

In [12]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_integer)

In [13]:
char_dataset

<TensorSliceDataset shapes: (), types: tf.int64>

In [14]:
for i in char_dataset.take(10):
    print(index2char[i.numpy()])

|
ب
ر
خ
ی
ز
 
ب
ت
ا


In [16]:
sequences = char_dataset.batch(101, drop_remainder=True)
sequences

<BatchDataset shapes: (101,), types: tf.int64>

In [17]:
for i in sequences.take(3):
    print('--->', ''.join(index2char[i.numpy()]))

---> |برخیز بتا بیا ز بهر دل ما
|حل کن به جمال خویشتن مشکل ما
|یک کوزه شراب تا به هم نوش کنیم
|زآن پیش که 
---> کوزه ها کنند از گل ما
|چون عهده نمی شود کسی فردا را
|حالی خوش دار این دل پر سودا را
|می نوش به ماهتاب
--->  ای ماه که ماه
|بسیار بتابد و نیابد ما را
|قرآن که مهین کلام خوانند آن را
|گه گاه نه بر دوام خوانند آ


In [18]:
def sit(batch):
    input_text = batch[:-1]
    target_text = batch[1:]
    return input_text, target_text
dataset = sequences.map(sit)

In [19]:
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [20]:
for i in dataset.take(1):
    print(''.join(index2char[i[0].numpy()]))
    print(''.join(index2char[i[1].numpy()]))

|برخیز بتا بیا ز بهر دل ما
|حل کن به جمال خویشتن مشکل ما
|یک کوزه شراب تا به هم نوش کنیم
|زآن پیش که
برخیز بتا بیا ز بهر دل ما
|حل کن به جمال خویشتن مشکل ما
|یک کوزه شراب تا به هم نوش کنیم
|زآن پیش که 


In [21]:
dataset = dataset.batch(64, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [47]:
len(vocabolaries)

47

In [53]:
vocabolary_size = len(vocabolaries)
embedding_dim = 256
rnn_units = 1024

In [54]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocabolary_size, embedding_dim),
    tf.keras.layers.GRU(rnn_units, return_sequences=True),
    tf.keras.layers.Dense(vocabolary_size)
])

In [55]:
for input_text, target_text in dataset.take(1):
    output = model.predict(input_text)
    print(output[0])

[[-0.00091032 -0.01065248 -0.00276685 ... -0.00311616 -0.01000043
   0.00622381]
 [ 0.01249189 -0.01199279 -0.00528852 ...  0.00613781 -0.00536665
  -0.00217031]
 [ 0.00802283 -0.00017949  0.00494839 ...  0.00456399 -0.01974823
   0.00710435]
 ...
 [ 0.01679767 -0.00580759 -0.00136256 ... -0.01375992 -0.00782239
  -0.0134883 ]
 [ 0.00117025 -0.0101225  -0.00244823 ... -0.00958827 -0.00711462
  -0.01956189]
 [ 0.0088116  -0.01171549  0.00034929 ... -0.00574466  0.01293677
   0.00230383]]


In [56]:
si = tf.random.categorical(output[0], num_samples=1)
si

<tf.Tensor: shape=(100, 1), dtype=int64, numpy=
array([[ 4],
       [26],
       [25],
       [37],
       [17],
       [45],
       [19],
       [27],
       [42],
       [37],
       [36],
       [21],
       [18],
       [37],
       [ 6],
       [22],
       [11],
       [ 7],
       [38],
       [41],
       [35],
       [37],
       [23],
       [ 5],
       [ 2],
       [20],
       [ 8],
       [33],
       [26],
       [ 4],
       [24],
       [36],
       [37],
       [ 0],
       [32],
       [11],
       [11],
       [34],
       [21],
       [22],
       [ 6],
       [14],
       [23],
       [37],
       [36],
       [45],
       [19],
       [ 7],
       [14],
       [17],
       [44],
       [31],
       [11],
       [26],
       [ 9],
       [31],
       [12],
       [13],
       [14],
       [41],
       [29],
       [18],
       [ 0],
       [ 6],
       [17],
       [23],
       [26],
       [37],
       [39],
       [12],
       [20],
       [ 6],
       [42],
   

In [57]:
tf.squeeze(si, axis=-1).numpy()

array([ 4, 26, 25, 37, 17, 45, 19, 27, 42, 37, 36, 21, 18, 37,  6, 22, 11,
        7, 38, 41, 35, 37, 23,  5,  2, 20,  8, 33, 26,  4, 24, 36, 37,  0,
       32, 11, 11, 34, 21, 22,  6, 14, 23, 37, 36, 45, 19,  7, 14, 17, 44,
       31, 11, 26,  9, 31, 12, 13, 14, 41, 29, 18,  0,  6, 17, 23, 26, 37,
       39, 12, 20,  6, 42, 10, 19,  0, 38, 45,  8,  5, 18, 36, 38, 25, 12,
       32,  1, 22, 16, 46, 29, 22, 26, 37, 23, 23, 16,  0, 24, 24])

In [58]:
''.join(index2char[tf.squeeze(si, axis=-1).numpy()])

'؟ظطٌدگرعچًٌسذٌآشةؤَپوٌصء!زئنظ؟ضًٌ\nمةةهسشآجصًٌگرؤجدکلةظالتثجپفذ\nآدصظٌّتزآچبر\nَگئءذًَطتم شخیفشظٌصصخ\nضض'

In [59]:
output[0][0]

array([-9.1031846e-04, -1.0652477e-02, -2.7668499e-03,  8.0151083e-03,
        6.7482088e-03,  1.3226552e-02,  8.8221245e-03,  1.1680550e-03,
        1.6145175e-05,  1.1015472e-02, -7.8903488e-04, -2.7878855e-03,
        4.0753628e-05, -1.3159969e-03,  7.9447832e-03,  2.3272196e-03,
       -4.6412339e-03,  3.2124023e-03, -1.7258890e-02,  1.4421007e-02,
       -6.5471772e-03,  3.4239872e-03,  1.3249269e-02,  7.7808877e-03,
       -3.5939631e-03,  8.9376196e-03, -5.5991383e-03, -3.9480301e-04,
        6.0380795e-03, -4.8489720e-03, -8.3467979e-03,  1.1604073e-02,
       -1.0743539e-02,  1.0800222e-03, -7.3164375e-03,  5.6805396e-03,
        3.8239043e-03, -1.4297187e-03, -1.7495240e-03,  1.6684338e-02,
       -1.0629360e-02, -2.9962743e-04, -5.4808506e-03, -5.1780995e-03,
       -3.1161553e-03, -1.0000431e-02,  6.2238057e-03], dtype=float32)

In [60]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 256)         12032     
_________________________________________________________________
gru_4 (GRU)                  (None, None, 1024)        3938304   
_________________________________________________________________
dense_4 (Dense)              (None, None, 47)          48175     
Total params: 3,998,511
Trainable params: 3,998,511
Non-trainable params: 0
_________________________________________________________________


In [61]:
def loss_f(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
model.compile(optimizer='adam', loss=loss_f)

In [62]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='khayyammolana/checkpoints', save_weights_only=True)

In [77]:
history = model.fit(dataset, epochs=100, callbacks=[checkpoint])

Epoch 1/10
11/11 [==============================] - 6s 570ms/step - loss: 4.0833
Epoch 2/10
11/11 [==============================] - 7s 591ms/step - loss: 3.6942
Epoch 3/10
11/11 [==============================] - 6s 575ms/step - loss: 3.6508
Epoch 4/10
11/11 [==============================] - 6s 587ms/step - loss: 3.3799
Epoch 5/10
11/11 [==============================] - 6s 588ms/step - loss: 3.0961
Epoch 6/10
11/11 [==============================] - 7s 594ms/step - loss: 3.0173
Epoch 7/10
11/11 [==============================] - 6s 504ms/step - loss: 2.9842
Epoch 8/10
11/11 [==============================] - 6s 504ms/step - loss: 2.9480
Epoch 9/10
11/11 [==============================] - 6s 540ms/step - loss: 2.9029
Epoch 10/10
11/11 [==============================] - 6s 517ms/step - loss: 2.8414


In [63]:
tf.train.latest_checkpoint('khayyammolana/')

'khayyammolana/checkpoints'

In [64]:
model_2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocabolary_size, embedding_dim),
    tf.keras.layers.GRU(rnn_units, return_sequences=True),
    tf.keras.layers.Dense(vocabolary_size)
])

In [65]:
model_2.load_weights(tf.train.latest_checkpoint('khayyammolana'))

In [66]:
model_2.build(tf.TensorShape([1, None]))

In [67]:
model_2.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 256)         12032     
_________________________________________________________________
gru_5 (GRU)                  (None, None, 1024)        3938304   
_________________________________________________________________
dense_5 (Dense)              (None, None, 47)          48175     
Total params: 3,998,511
Trainable params: 3,998,511
Non-trainable params: 0
_________________________________________________________________


In [94]:
num_generate = 1000
first_string = 'به نام خداوند جان و خرد'
input_eval = [char2index[s] for s in first_string]
input_eval = tf.expand_dims(input_eval, 0)
input_eval

<tf.Tensor: shape=(1, 23), dtype=int32, numpy=
array([[10, 34,  1, 33,  9, 32,  1, 16, 17,  9, 35, 33, 17,  1, 14,  9,
        33,  1, 35,  1, 16, 19, 17]], dtype=int32)>

In [87]:
model_2.reset_states()

In [95]:
text_generated = ['به نام خداوند جان و خرد']
for i in range(500):
    predictions = model_2.predict(input_eval)
    predictions = tf.squeeze(predictions, 0)
#     predicted_ids = tf.random.categorical(predictions, num_samples=1).numpy()
    predicted_ids = np.array(predictions.numpy()).argmax(axis=1).reshape(-1, 1)[-1][0]
    print(predicted_ids)
    message = np.append(input_eval[0].numpy(), predicted_ids)[1:]
    input_eval = tf.expand_dims(message, 0)
    text_generated.append(index2char[predicted_ids])


0
3
17
19
1
10
46
9
33
1
32
9
33
17
1
9
20
1
9
10
17
46
31
1
35
1
45
32
0
3
33
9
30
34
40
1
14
21
32
1
35
31
46
1
19
9
1
10
21
1
10
17
46
17
0
3
42
35
33
1
10
32
9
33
17
1
9
20
1
16
10
19
1
6
28
9
20
1
44
19
17
0
3
45
9
34
1
10
28
17
1
9
20
1
17
22
32
33
9
33
1
9
46
33
1
19
9
1
22
33
9
21
0
3
42
35
33
1
10
17
9
33
17
1
42
19
16
1
19
9
1
32
33
1
17
19
1
17
31
46
0
3
45
19
1
33
32
9
20
1
35
1
19
35
20
1
32
46
1
6
19
17
1
21
16
33
0
3
42
35
33
1
33
46
44
1
35
1
10
17
22
1
20
46
33
1
33
9
33
1
42
19
9
0
3
34
19
1
42
34
1
10
19
1
12
21
35
46
19
1
42
46
33
34
1
31
14
12
1
35
1
21
16
9
33
1
35
1
19
35
15
1
34
21
12
0
3
6
33
1
9
13
19
1
32
46
1
10
9
20
46
1
17
16
35
46
22
1
35
1
17
22
9
17
0
3
17
19
1
32
46
9
33
1
16
35
9
10
1
14
21
12
1
35
1
45
9
34
1
6
33
1
16
31
46
1
14
35
1
10
19
17
1
9
46
33
0
3
41
46
22
1
6
19
17
1
30
19
23
1
17
46
33
1
19
9
1
35
9
1
19
34
9
33
0
3
42
22
32
34
40
1
16
35
33
1
19
9
1
10
29
33
1
9
20
1
19
46
45
1
9
35
0
3
10
21
1
45
19
9
20
1
6
10
21
12
1
33
30
22
9
33
1
3

In [96]:
''.join(text_generated).split('\n')

['به نام خداوند جان و خرد',
 '|در بیان ماند از ابدیل و گم',
 '|ناقهٔ جسم ولی را بس بدید',
 '|چون بماند از خبر آغاز کرد',
 '|گاه بغد از دشمنان این را شناس',
 '|چون بداند چرخ را من در دلی',
 '|گر نماز و روز می آرد سخن',
 '|چون نیک و بدش زین نان چرا',
 '|هر چه بر تسویر چینه لجت و سخان و روح هست',
 '|آن اثر می بازی دخویش و دشاد',
 '|در میان خواب جست و گاه آن خلی جو برد این',
 '|پیش آرد قرص دین را وا رهان',
 '|چشمهٔ خون را بفن از ریگ او',
 '|بس گراز آبست نقشان و ثباب',
 '|چونک آواز خلق از بیخ و نو',
 '|با شما از خلق این سوزی و دست',
 '|در بیابانهای بی چندی شکست',
 '|اندر آ مادر مده دادان یکی']

In [97]:
for i in ''.join(text_generated).split('\n'):
    print(i)

به نام خداوند جان و خرد
|در بیان ماند از ابدیل و گم
|ناقهٔ جسم ولی را بس بدید
|چون بماند از خبر آغاز کرد
|گاه بغد از دشمنان این را شناس
|چون بداند چرخ را من در دلی
|گر نماز و روز می آرد سخن
|چون نیک و بدش زین نان چرا
|هر چه بر تسویر چینه لجت و سخان و روح هست
|آن اثر می بازی دخویش و دشاد
|در میان خواب جست و گاه آن خلی جو برد این
|پیش آرد قرص دین را وا رهان
|چشمهٔ خون را بفن از ریگ او
|بس گراز آبست نقشان و ثباب
|چونک آواز خلق از بیخ و نو
|با شما از خلق این سوزی و دست
|در بیابانهای بی چندی شکست
|اندر آ مادر مده دادان یکی
